In [3]:
import json
import os
import numpy as np
from copy import deepcopy
def min_max(input_list):# MIn Max 정규화 함수
    max_num=max(input_list)
    min_num=min(input_list)
    for i in range(len(input_list)):
        input_list[i]= (input_list[i] - min_num )/ (max_num - min_num)
    return input_list

path_dir = './Desktop/dance/골반'# 부위별 파일
 
file_list = os.listdir(path_dir)
print(file_list)
label_list=[]# 라벨링 숫자를 담는 리스트
final_list=[]# 최종적으로 클러스터링 진행할 리스트
missing_position=[]# open pose에서 인식하지 못한 포인트

for j in range (len(file_list)):
    new_path = path_dir + '/' + file_list[j]
    new_file_list = os.listdir(new_path)
    new_path_2=new_path+'/'+new_file_list[0]#문서 파일로 들어감
    new_file_list2=os.listdir(new_path_2)
    new_list=[] # 한번의 동작 프레임을 담는 리스트
    partial_list=[] #한번의 동작 프레임을 담되, 부위별로 focusing할 key point만 담는 리스트
    for i in new_file_list2:
        new_path_3= new_path_2+'/'+i
        b=i.split('_')
        if (b[3]=='000000000000' and i !=new_file_list2[0]):# 프레임이 새로 시작할때, 이전 프레임까지의 동작을 리스트에 담음
            final_list.append(partial_list)
            new_list=[]
            patial_list=[]
            label_list.append(j)#해당 동작 리스트에 대한 라벨을 리스트에 담음

        with open(new_path_3, 'r') as f:# json 파일 읽어서 key point 좌표 데이터 획득
            json_data = json.load(f)
        a=json_data['people'][0]['pose_keypoints_2d']
        x_coord=[]
        y_coord=[]
        b=np.zeros(50).tolist()# 총 75중, 25개의 신뢰도 데이터 제외하고 50개의 xy좌표 데이터만 사용
        

        for k in range(75):
            if k%3==1:# y 좌표, 평균값 400 
                if a[k]==0:# key point가 인식되지 않아, 좌표값이 0일때, 이전값 사용
                    if len(new_list)==0:
                        a[k]=400
                    else:    
                        position=int((2*k+1)/3)
                        b[position]=new_list[len(new_list)-1][position]
                        missing_position.append(position)
                        a[k]=400        
                a[k]-=a[25] # key point 좌표가 중앙에 모이도록 골반 y좌표만큼 뺌
                y_coord.append(a[k])
            if k%3==0:# x 좌표, 평균값 900
                if a[k]==0:# key point가 인식되지 않아, 좌표값이 0일때, 이전값 사용
                    if len(new_list)==0:
                        a[k]=900
                    else:    
                        position_2= int(2*k/3)                   
                        b[position_2]=new_list[len(new_list)-1][position_2]
                        missing_position.append(position_2)
                        a[k]=900
                a[k]-=a[24] # key point 좌표가 중앙에 모이도록 골반 x좌표만큼 뺌
                x_coord.append(a[k])   
        x_coord=deepcopy(min_max(x_coord)) # min-max normalization
        y_coord=deepcopy(min_max(y_coord))
        
        for k in range(50):
            if k in missing_position:#missing point 재외하고 normalize된 좌표값 리스트에 담음
                continue
            if k%2==0:
                b[k]=x_coord[int(k/2)]
            if k%2==1:
                b[k]=y_coord[int((k-1)/2)]
        key_point_num=[8,9,12]#부위별로 사용할 key point  ex) 골반은 8,9,12
        c=[] 
        for m in key_point_num:#부위별로 지정한 key point만 리스트에 담음 
            c.append(b[2*m])
            c.append(b[2*m+1])
        partial_list.append(c)
        new_list.append(b)
        missing_position=[]
        
        if i==new_file_list2[len(new_file_list2)-1]:
            final_list.append(partial_list)
            label_list.append(j)
        

import tslearn
from tslearn.clustering import TimeSeriesKMeans 
import numpy as np

X_bis = tslearn.utils.to_time_series_dataset(final_list)

km = TimeSeriesKMeans(n_clusters=len(file_list), metric="dtw", max_iter=10,random_state=0)#클러스터링 진행

labels = km.fit_predict(X_bis)

prediction=labels.tolist()
print(label_list)
print(prediction)

['골반_동적_돌리기-관절(rotation)', '골반_동적_들이밀어내밀기(push in and out)', '골반_동적_밀기(push)', '골반_동적_빙그르르, 빙글빙글(twirl)', '골반_동적_씰룩쌜룩', '골반_동적_흔들(-흔들)(swayingly, rockingly)']
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5]
[2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 0, 0, 5, 1, 1, 1, 1, 1, 1, 1, 1]
